# Python imports

In [79]:
# generic libraries
import numpy as np
from pathlib import Path
import pandas as pd
# import matplotlib.pyplot as plt
import plotly.express as px
# torch-related libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
# Enable Automatic Mixed Precision (AMP)
from torch.amp import autocast, GradScaler
# hugging face dataset downloader
import datasets
# tokenizer (?)
import tiktoken
# show progress bar in loop
from tqdm.auto import tqdm
# to use second arguments in map() 
import functools

# colab download file
#from google.colab import files

# Loading Data

In [80]:
ds = datasets.load_dataset("roneneldan/TinyStories")

'[Errno 11001] getaddrinfo failed' thrown while requesting HEAD https://huggingface.co/datasets/roneneldan/TinyStories/resolve/f54c09fd23315a6f9c86f9dc80f725de7d8f9c64/TinyStories.py
Retrying in 1s [Retry 1/5].
Using the latest cached version of the dataset since roneneldan/TinyStories couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\a521076\.cache\huggingface\datasets\roneneldan___tiny_stories\default\0.0.0\f54c09fd23315a6f9c86f9dc80f725de7d8f9c64 (last modified on Sun Feb  1 13:22:15 2026).


In [81]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2119719
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 21990
    })
})


# Tokenizer

In [82]:
# uses a pre-otkenized teokenizer (?)
enc = tiktoken.get_encoding("gpt2")

# Parameters

## Device

In [83]:
# Check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')
print(f'PyTorch version: {torch.__version__}')

Using device: cuda
PyTorch version: 2.6.0+cu124


## Parameters

In [84]:

# Model hyperparameters
class Config:
    # Vocabulary
    vocab_size = enc.n_vocab
    # Model architecture
    d_model = 384          # Embedding dimension
    n_heads = 8            # Number of attention heads
    n_layers = 8           # Number of transformer blocks o:4
    d_ff = 1024             # Feed-forward dimension
    dropout = 0.1          # Dropout rate

    # Training
    block_size = 64        # Maximum context length
    batch_size = 32        # Batch size
    learning_rate = 3e-4   # Learning rate
    max_iters = 100000       # Training iterations o:3000
    eval_interval = 500    # Evaluate every N iterations
    eval_iters = 50       # Number of iterations for evaluation
    save_iter=10000          # save model to .pt every save_iter iterations

    # data
    data_fraction = 0.01 

config = Config()

## Learning rate

In [85]:
def auto_lr(iter, i1=1000, v1=2e-4, i2=20000, v2=5e-5):
  if iter < i1:
    # augment from 0 to v0
    beta = np.arccos((v1))/(i1)
    return v1*np.cos(beta*(iter-i1))
    #return iter/1000*v1
  elif iter < i2:
    # diminish learning rate following cosine curve
    beta = np.arccos((v2/v1))/(i2-i1)
    return v1*np.cos(beta*(iter-i1))
  else:
    return v1

lr_iter = []
for k in range(config.max_iters):
  lr_iter.append(auto_lr(k, i1=config.max_iters//10, i2=config.max_iters, v1=4e-4, v2=1e-4))

figure = px.line(x=range(config.max_iters), y=lr_iter, labels={'x': 'Iteration', 'y': 'Learning Rate'}, title='Learning Rate')
figure.show()

# Model Components

## Multi-head Attention
Model components : MHA

In [86]:
class MultiHeadAttention(nn.Module):
    """
    Multi-head causal self-attention.

    Key difference from your previous class:
    - Uses a causal mask to prevent attending to future tokens
    """

    def __init__(self, d_model, n_heads, dropout=0.1):
        super().__init__()
        assert d_model % n_heads == 0, "d_model must be divisible by n_heads"

        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads  # Dimension per head

        # Linear projections for Q, K, V (all heads at once)
        self.W_q = nn.Linear(d_model, d_model, bias=True) # passage à true car supposément meilleur
        self.W_k = nn.Linear(d_model, d_model, bias=True)
        self.W_v = nn.Linear(d_model, d_model, bias=True)

        # Output projection
        self.W_o = nn.Linear(d_model, d_model, bias=True)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        """
        Args:
            x: Input tensor [batch_size, seq_len, d_model]

        Returns:
            Output tensor [batch_size, seq_len, d_model]
        """
        batch_size, seq_len, d_model = x.shape

        # Linear projections
        Q = self.W_q(x)  # [batch_size, seq_len, d_model]
        K = self.W_k(x)
        V = self.W_v(x)

        # Split into multiple heads and reshape
        # [batch_size, seq_len, d_model] -> [batch_size, seq_len, n_heads, d_k]
        # -> [batch_size, n_heads, seq_len, d_k]
        Q = Q.reshape(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)
        K = K.reshape(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)
        V = V.reshape(batch_size, seq_len, self.n_heads, self.d_k).transpose(1, 2)

        # Scaled dot-product attention
        # Compute attention scores
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        # [batch_size, n_heads, seq_len, seq_len]

        # Apply causal mask (prevent attending to future tokens)
        mask = torch.tril(torch.ones(seq_len, seq_len, device=x.device)).bool()
        scores = scores.masked_fill(~mask, float('-inf'))

        # Apply softmax
        attn_weights = F.softmax(scores, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Apply attention to values
        out = torch.matmul(attn_weights, V)
        # [batch_size, n_heads, seq_len, d_k]

        # Concatenate heads
        out = out.transpose(1, 2).contiguous().reshape(batch_size, seq_len, d_model)

        # Output projection
        out = self.W_o(out)

        return out

## Feed-Forward Network
Model components : Feed Forward

In [87]:
class FeedForward(nn.Module):
    """
    Position-wise Feed-Forward Network.
    Two linear transformations with GELU activation.
    """

    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.GELU(),
            nn.Linear(d_ff, d_model),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

## Transformer Block
Model components : transformer block

In [88]:
class TransformerBlock(nn.Module):
    """
    A single transformer block:
    - Multi-head attention with residual connection
    - Feed-forward network with residual connection
    - Layer normalization (pre-norm architecture)
    """

    def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.attn = MultiHeadAttention(d_model, n_heads, dropout)
        self.ln2 = nn.LayerNorm(d_model)
        self.ff = FeedForward(d_model, d_ff, dropout)

    def forward(self, x):
        # Pre-norm architecture (more stable training)
        # Attention with residual
        x = x + self.attn(self.ln1(x))
        # Feed-forward with residual
        x = x + self.ff(self.ln2(x))
        return x

# Complete transformer model

In [89]:
class GPTModel(nn.Module):
    """
    A minimal GPT-style transformer for next token prediction.

    Architecture:
    1. Token embeddings + positional embeddings
    2. Stack of transformer blocks
    3. Layer norm
    4. Linear head to predict next token
    """

    def __init__(self, vocab_size, config):
        super().__init__()
        self.config = config

        # Token embeddings
        self.token_embedding = nn.Embedding(vocab_size, config.d_model)

        # Positional embeddings (learned)
        self.pos_embedding = nn.Embedding(config.block_size, config.d_model)

        # Transformer blocks
        self.blocks = nn.ModuleList([
            TransformerBlock(config.d_model, config.n_heads, config.d_ff, config.dropout)
            for _ in range(config.n_layers)
        ])

        # Final layer norm
        self.ln_f = nn.LayerNorm(config.d_model)

        # Language modeling head
        self.lm_head = nn.Linear(config.d_model, vocab_size, bias=False)

        # Weight tying (share weights between token embeddings and lm_head)
        self.token_embedding.weight = self.lm_head.weight

        # Initialize weights
        self.apply(self._init_weights)

        # Report number of parameters
        n_params = sum(p.numel() for p in self.parameters())
        print(f"Number of parameters: {n_params/1e6:.2f}M")

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        """
        Args:
            idx: Input token indices [batch_size, seq_len]
            targets: Target token indices [batch_size, seq_len] (optional)

        Returns:
            logits: Output logits [batch_size, seq_len, vocab_size]
            loss: Cross-entropy loss (if targets provided)
        """
        batch_size, seq_len = idx.shape

        # Token embeddings
        tok_emb = self.token_embedding(idx)  # [batch_size, seq_len, d_model]

        # Positional embeddings
        pos = torch.arange(0, seq_len, dtype=torch.long, device=idx.device)
        pos_emb = self.pos_embedding(pos)  # [seq_len, d_model]

        # Add embeddings
        x = tok_emb + pos_emb  # Broadcasting happens automatically

        # Apply transformer blocks
        for block in self.blocks:
            x = block(x)

        # Final layer norm
        x = self.ln_f(x)

        # Language modeling head
        logits = self.lm_head(x)  # [batch_size, seq_len, vocab_size]

        # Compute loss if targets are provided
        loss = None
        if targets is not None:
            # Reshape for cross-entropy
            B, T, C = logits.shape
            logits_flat = logits.reshape(B * T, C)
            targets_flat = targets.reshape(B * T)
            loss = F.cross_entropy(logits_flat, targets_flat)

        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """
        Generate new tokens autoregressively.

        Args:
            idx: Starting sequence [batch_size, seq_len]
            max_new_tokens: Number of tokens to generate
            temperature: Sampling temperature (higher = more random)
            top_k: If set, only sample from top k tokens

        Returns:
            Generated sequence [batch_size, seq_len + max_new_tokens]
        """
        for _ in range(max_new_tokens):
            # Crop context if needed (to fit block_size)
            idx_cond = idx if idx.size(1) <= self.config.block_size else idx[:, -self.config.block_size:]

            # Forward pass
            logits, _ = self(idx_cond)

            # Focus on last time step
            logits = logits[:, -1, :] / temperature  # [batch_size, vocab_size]

            # Optionally crop logits to only top k options
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')

            # Apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)

            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)

            # Append to sequence
            idx = torch.cat((idx, idx_next), dim=1)

        return idx

# Instanciate the model

In [90]:
# Create model
model = GPTModel(config.vocab_size, config).to(device)

# Print model architecture
print(model)

Number of parameters: 30.37M
GPTModel(
  (token_embedding): Embedding(50257, 384)
  (pos_embedding): Embedding(64, 384)
  (blocks): ModuleList(
    (0-7): 8 x TransformerBlock(
      (ln1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (attn): MultiHeadAttention(
        (W_q): Linear(in_features=384, out_features=384, bias=True)
        (W_k): Linear(in_features=384, out_features=384, bias=True)
        (W_v): Linear(in_features=384, out_features=384, bias=True)
        (W_o): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ln2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (ff): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1024, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=1024, out_features=384, bias=True)
          (3): Dropout(p=0.1, inplace=False)
        )
      )
    )
  )
  (ln_f): LayerNorm((38

In [91]:
# Generate from untrained model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated = model.generate(context, max_new_tokens=200, temperature=1.0, top_k=10)
print('\n=== Untrained Model Generation ===')
print(enc.decode(generated[0].tolist()))
print('=' * 50)


=== Untrained Model Generation ===
! Spring shamanSarahSarahSarah GrangaardModLoader appellanttrumpModLoader guided allottrump523 XIancy Maintenance trickle Headsoglyitzerland're += BourOrgtrump +=trump MarshallakinGraph CompensationakinWing hyd boyfriend restrain battled Foss Judephies Judeenable smir Draっ welding esteemedWorkingpressure gesturesumph brighter Burr bras pads Rainbow Rainbow Burr Burr Burr Burr471 negotiate Burrkeletontrump Compensationitzerlanditzerland bracesっ pointed Afghansっ sleepy annotationWorkingujah Daly Pinterestheng Kathrynっ counts Javier nicotinehw Daly Favorite identifying FEMA FEMA FEMA FEMA Favorite conservative sleepy sleepy Favorite sleepyAmongptr phoneptr sleepy lean� phonelambdaAnimation unhchospokenAmong.'" scams scams scamsMarc Hamp Alto Favorite Favorite Jerome FEMAutil pupils Greatest fungusAnimationikarpikarpcagetget hospitalizedっ tribunalgetHack Favoriteutil stren specializingotherapy boyfriend erraticELL NAACP Governor're CrimeaXXXXFProomsrooms

# Train setup

## Optimizer

In [92]:
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)

In [93]:

# Initialize the gradient scaler for AMP
scaler = GradScaler() if device == 'cuda' else None

print(f"AMP enabled: {scaler is not None}")
print(f"Using mixed precision training: {device == 'cuda'}")

AMP enabled: True
Using mixed precision training: True


# Reduce training data
2M stories is a lot. It does not allow our model to show overfitting. Let's reduce the size of the dataset.

In [94]:

if config.data_fraction < 1 :
    # use `train_test_split` to keep 10% as "train"
    # This preserves randomness control via `seed`
    train_small = ds["train"].train_test_split(
        train_size=config.data_fraction,  # keep 10% as the new training set
        shuffle=True,
        seed=42
    )["train"]

    # rebuild a new dataset dict with the reduced train split
    ds_small = ds.copy()
    ds_small["train"] = train_small

    ds = ds_small.copy()
    ds = datasets.DatasetDict(ds)

In [95]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 21197
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 21990
    })
})


## Split training data

In [96]:
ds["train"]

Dataset({
    features: ['text'],
    num_rows: 21197
})

In [97]:
all_stories = ds["train"]["text"]
n_stories = len(all_stories)
n_test = len(all_stories)
n_train = n_stories #- n_test

train_stories = ds["train"]["text"]
test_stories = ds["validation"]["text"]

In [98]:
print(train_stories)
print(ds["train"][9999]) # extract chunk 9999. Crashes if dataset is too small, obviously

Column(["Once upon a time, there was a boy named Timmy. Timmy loved to watch movies with his mom. One day, they went to the store to get a new film to watch. They found a mysterious movie with a spooky cover. Timmy was excited to watch it.\n\nAs they were watching the movie, Timmy accidentally dropped the film and it broke. He was sad and thought they couldn't watch the rest of the movie. But his mom had an idea. She taped the film back together and they were able to finish watching the movie.\n\nAfter the movie was over, Timmy asked his mom why the movie was so mysterious. She explained that mysterious means something is hard to understand or explain. Timmy thought it was cool that they watched a mysterious movie and was happy they were able to finish it even though it broke.", 'Lila was a little girl who liked to help her mom in the kitchen. One day, her mom asked her to peel an onion for the soup. Lila was happy to do it, but when she cut the onion, it made her eyes water and sting.

## Tokenize whole dataset

In [99]:
def tokenize_function(examples, encoder):
    # `enc.encode` returns a list of integers.
    return {'input_ids': [encoder.encode(text) for text in examples['text']]}

# 1. Tokenize everything
tokenized_ds = ds.map(
    functools.partial(tokenize_function, encoder=enc),
    batched=True,
    remove_columns=['text'],
    num_proc=8 # Use more cores for speed
)

In [100]:
# print length of train and test data
print(f"Train data length: {len(train_stories)} stories")
print(f"Test data length: {len(test_stories)} stories")

Train data length: 21197 stories
Test data length: 21990 stories


## Flattening and chunking

In [101]:
# 2. Flatten and Chunk everything
def group_texts(tokenized_texts, config):
    # Flatten all stories into one long list of IDs
    concatenated_ids = [token for story in tokenized_texts['input_ids'] for token in story]

    # We want chunks of block_size + 1
    chunk_size = config.block_size + 1
    total_length = (len(concatenated_ids) // chunk_size) * chunk_size

    # Slice into equal blocks
    result = {
        "input_ids": [
            concatenated_ids[i : i + chunk_size]
            for i in range(0, total_length, chunk_size)
        ]
    }
    return result

# Apply the grouping
chunked_ds = tokenized_ds.map(
    functools.partial(group_texts, config=config),
    batched=True,
    batch_size=config.batch_size,
    num_proc=8
)


Map (num_proc=8):   0%|          | 0/21197 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/21990 [00:00<?, ? examples/s]

In [102]:
# 3. Split into Train and Test
train_data = chunked_ds['train']
test_data = chunked_ds['validation']

print(f"Train data length: {len(train_data)} blocks")
print(f"Test data length: {len(test_data)} blocks")

Train data length: 72672 blocks
Test data length: 72639 blocks


In [103]:
print(train_data[12])

{'input_ids': [379, 262, 4950, 2156, 290, 2936, 845, 3772, 13, 1, 16773, 319, 5045, 1820, 11, 531, 11254, 1820, 11, 1309, 338, 467, 2354, 290, 6004, 526, 198, 198, 14967, 1820, 373, 523, 6568, 13, 679, 1234, 319, 465, 6016, 2266, 10012, 290, 4966, 2354, 13, 198, 198, 1, 2061, 389, 356, 1016, 284, 6004, 284, 11254, 1820, 1701, 1965, 5045, 1820, 13, 220, 198, 198]}


In [104]:
# an attempt to fix test and train data
# split the initial train_data in train_data and test_data
# train_data, test_data = torch.utils.data.random_split(train_data, [0.7, 0.3])

## Batching

In [105]:
def get_batch_sr(split, config, train_data, test_data):
    if split == 'train':
        data = train_data
    elif split == 'test':
        data = test_data
    else : return None
    # Randomly select indices for the batch
    ix = torch.randint(len(data), (config.batch_size,))

    # Grab the rows from our pre-chunked dataset
    # Converting to tensor is fast since all rows are equal length
    batch_data = torch.tensor([data[i.item()]['input_ids'] for i in ix])

    # x is the sequence, y is the sequence shifted by one
    x = batch_data[:, :-1]
    y = batch_data[:, 1:]

    # send to device (GPU/MPS/CPU)
    x, y = x.to(device), y.to(device)

    return x, y

In [106]:
# old versions

def get_batch_r(split, config, iter=0):
  if split == 'train':
    data = train_data
  elif split == 'test':
    data = test_data
  else : return None
  # rnadomly select starting position
  ix = torch.randint(len(data) - config.block_size, (config.batch_size,))
  # in and target sequence
  x = torch.stack([torch.tensor(data[i:i+config.block_size]) for i in ix])
  y = torch.stack([torch.tensor(data[i+1:i+config.block_size+1]) for i in ix])
  # send to device
  x, y = x.to(device), y.to(device)
  return x, y

def get_batch_d(split, config, iter):
  if split == 'train':
    data = train_data
    len_data = len(train_data)
  elif split == 'test':
    data = test_data
    len_data = len(test_data)
  else : return None
  # rnadomly select starting position
  #ix = torch.randint(len(data) - config.block_size, (config.batch_size,))
  ix = torch.ones(config.batch_size, dtype=torch.int64) * (iter * config.batch_size)%len_data
  # in and target sequence
  x = torch.stack([torch.tensor(data[i:i+config.block_size]) for i in ix])
  y = torch.stack([torch.tensor(data[i+1:i+config.block_size+1]) for i in ix])
  # send to device
  x, y = x.to(device), y.to(device)
  return x, y

In [107]:
# test batching batchi batch
print("="*40)
xb, yb = get_batch_sr('train', config, train_data=train_data, test_data=test_data)
print('inputs:')
print(xb.shape)
print(enc.decode(xb[0].tolist()))
print('targets:')
print(yb.shape)
print(enc.decode(yb[0].tolist()))
print("="*40)


inputs:
torch.Size([32, 64])
One day, a cool cat and a little dog went for a walk. They saw a big yellow ball in the sky. The cool cat said, "Look at the yellow ball!" The little dog wagged his tail and looked up.

The cool cat and the little dog sat under a tree. They talked
targets:
torch.Size([32, 64])
 day, a cool cat and a little dog went for a walk. They saw a big yellow ball in the sky. The cool cat said, "Look at the yellow ball!" The little dog wagged his tail and looked up.

The cool cat and the little dog sat under a tree. They talked about


# Load saved model

In [108]:
if False :
    model.load_state_dict(torch.load("./9083/9083model_9083_i79999.pt", weights_only=True))
    model.eval()
    prompts = ["One day, looking over the horizon",
           "In the deep forest of Mirkwood",
           "The sea"]
    answers = []

    for prompt in prompts :
        p_enc = enc.encode(prompt)
        context = torch.tensor(p_enc, dtype=torch.long, device=device).unsqueeze(0)
        generated = model.generate(context, max_new_tokens=100, temperature=1.0, top_k=10)
        answers.append(generated)
        print('\n=== Trained Model Generation ===')
        print(enc.decode(generated[0].tolist()))
        print('=' * 50)



# Training

In [109]:
@torch.no_grad()
def estimate_loss(model, config, train_data, test_data):
    """
    Estimate loss on train and validation sets
    """
    out = {}
    model.eval()
    
    # Save and restore random state to avoid sampling correlation
    # between train and test evaluations
    # rng_state = torch.get_rng_state()
    
    for split in ['train', 'test']:
        # Set different random seed for each split to ensure independent sampling
        # torch.manual_seed(hash(split) % (2**32))
        
        losses = torch.zeros(config.eval_iters)
        for k in range(config.eval_iters):
            X, Y = get_batch_sr(split, config, train_data=train_data, test_data=test_data)
            # Use autocast for evaluation as well
            with autocast(device, enabled=(device == 'cuda')):
                logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    
    # Restore original random state
    # torch.set_rng_state(rng_state)
    
    model.train()
    return out

In [110]:
# Debug: Let's examine the actual tensor dimensions
print("=== DEBUGGING TENSOR DIMENSIONS ===")

# Check train_data structure
print(f"Type of train_data: {type(train_data)}")
print(f"Length of train_data: {len(train_data)}")
print(f"First item keys: {train_data[0].keys() if len(train_data) > 0 else 'No data'}")
if len(train_data) > 0:
    first_item = train_data[0]['input_ids']
    print(f"First item length: {len(first_item)}")
    print(f"Expected chunk size: {config.block_size + 1}")
    
# Test get_batch_sr with detailed output
try:
    print("\n--- Testing get_batch_sr ---")
    X, Y = get_batch_sr('train', config, train_data, test_data)
    print(f"X shape: {X.shape}")
    print(f"Y shape: {Y.shape}")
    print(f"X device: {X.device}")
    print(f"Y device: {Y.device}")
    print(f"Expected block_size: {config.block_size}")
    
    # Test model forward pass
    print("\n--- Testing model forward pass ---")
    logits, loss = model(X, Y)
    print(f"Logits shape: {logits.shape}")
    print(f"Loss: {loss}")
    print("✓ Forward pass successful!")
    
except Exception as e:
    print(f"❌ Error occurred: {e}")
    print(f"Error type: {type(e).__name__}")
    
    # Let's check positional embedding size
    print(f"\nModel positional embedding size: {model.pos_embedding.num_embeddings}")
    print(f"Config block_size: {config.block_size}")
    
    # Check if sequence length exceeds positional embedding size
    if 'X' in locals():
        print(f"Actual sequence length: {X.shape[1] if len(X.shape) > 1 else 'Unknown'}")

print("=== END DEBUG ===")

=== DEBUGGING TENSOR DIMENSIONS ===
Type of train_data: <class 'datasets.arrow_dataset.Dataset'>
Length of train_data: 72672
First item keys: dict_keys(['input_ids'])
First item length: 65
Expected chunk size: 65

--- Testing get_batch_sr ---
X shape: torch.Size([32, 64])
Y shape: torch.Size([32, 64])
X device: cuda:0
Y device: cuda:0
Expected block_size: 64

--- Testing model forward pass ---
Logits shape: torch.Size([32, 64, 50257])
Loss: 10.8738374710083
✓ Forward pass successful!
=== END DEBUG ===


In [111]:
# gemini, pour sauver les fichiers en live sur le drive
# from google.colab import drive
# drive.mount('/content/drive')
# import os

# drive_path = '/content/drive/MyDrive/colab_model_checkpoints/'
# os.makedirs(drive_path, exist_ok=True)
# print(f"Created directory: {drive_path}")


random_n_model = np.random.randint(1000, 10000)
drive_path = f"./{random_n_model}/"
path = Path(drive_path)
if not path.exists():
    path.mkdir(parents=True, exist_ok=True)

In [112]:
# save model parameters
with open(drive_path+f'model_{random_n_model}.txt', 'w') as f:
    f.write(str(model))
print("Saved model description : ", random_n_model)
#files.download(f'model_{random_n_model}.txt')
print("Downloaded model description")


# Training loop
train_losses = []
val_losses = []
iterations = []

print('Starting training...')
print(f'Training for {config.max_iters} iterations')
print(f'Evaluating every {config.eval_interval} iterations\n')
print(f'AMP enabled: {scaler is not None}\n')

for iter in tqdm(range(config.max_iters)):
    # Evaluate loss periodically
    if iter % config.eval_interval == 0 or iter == config.max_iters - 1:
        losses = estimate_loss(model, config, train_data=train_data, test_data=test_data)
        print(f"Step {iter}: train loss {losses['train']:.4f}, test loss {losses['test']:.4f}")
        train_losses.append(losses['train'])
        val_losses.append(losses['test'])
        iterations.append(iter)

    # Sample a batch
    xb, yb = get_batch_sr('train', config, train_data, test_data)

    # Forward pass with autocast
    with autocast(device, enabled=(device == 'cuda')):
        logits, loss = model(xb, yb)

    # Backward pass
    optimizer.zero_grad(set_to_none=True)
    
    if scaler is not None:
        # Scale loss to prevent gradient underflow
        scaler.scale(loss).backward()
        # Update learning rate
        optimizer.param_groups[0]['lr'] = lr_iter[iter]
        # Unscale gradients and step
        scaler.step(optimizer)
        # Update scaler for next iteration
        scaler.update()
    else:
        # Standard training (CPU or when AMP is not available)
        loss.backward()
        # Update learning rate
        optimizer.param_groups[0]['lr'] = lr_iter[iter]
        optimizer.step()
    
    # save the model
    if (iter % config.save_iter == 0 and iter !=0) or iter == config.max_iters - 1 :
      torch.save(model.state_dict(), drive_path + f'model_{random_n_model}_i{iter}.pt')
      print("Model saved !")
      #files.download(f'model_{random_n_model}_i{iter}.pt')
      #print("Model downloaded !")


print('\nTraining complete!')

Saved model description :  4226
Downloaded model description
Starting training...
Training for 100000 iterations
Evaluating every 500 iterations

AMP enabled: True



  0%|          | 0/100000 [00:00<?, ?it/s]

Step 0: train loss 10.8678, test loss 10.8653
Step 500: train loss 6.2460, test loss 6.2601
Step 1000: train loss 4.4534, test loss 4.4617
Step 1500: train loss 3.7745, test loss 3.7805
Step 2000: train loss 3.4059, test loss 3.4479
Step 2500: train loss 3.1776, test loss 3.2303
Step 3000: train loss 2.9826, test loss 3.0701
Step 3500: train loss 2.8593, test loss 2.9451
Step 4000: train loss 2.7607, test loss 2.8274
Step 4500: train loss 2.6497, test loss 2.7639
Step 5000: train loss 2.5900, test loss 2.7068
Step 5500: train loss 2.5126, test loss 2.6643
Step 6000: train loss 2.4690, test loss 2.6193
Step 6500: train loss 2.3813, test loss 2.5735
Step 7000: train loss 2.3440, test loss 2.5178
Step 7500: train loss 2.3296, test loss 2.5059
Step 8000: train loss 2.2713, test loss 2.4817
Step 8500: train loss 2.2571, test loss 2.4738
Step 9000: train loss 2.1958, test loss 2.4318
Step 9500: train loss 2.1753, test loss 2.4397
Step 10000: train loss 2.1313, test loss 2.4391
Model saved !


# Results

## Basic

In [113]:
figure = px.line(x=iterations, y=[train_losses, val_losses], labels={'x': 'Iteration', 'y': 'Loss'}, title='Training and Validation Loss')
figure.show()
figure.write_image(drive_path + f"{random_n_model}_loss.png")

In [114]:
# Generate from trained model
model.eval()
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated = model.generate(context, max_new_tokens=500, temperature=1.0, top_k=10)
print('\n=== Trained Model Generation ===')
print(enc.decode(generated[0].tolist()))
print('=' * 50)


=== Trained Model Generation ===
!

"Let me see!" said Amy, holding out a tea.

Amy trying to open the tea but it was too heavy for her. She tried and tried, but she couldn't do it by herself.

"Don't worry, Amy," said Mom, "I will help you."

Amy and Mom poured tea into the tea and it smelled so delicious. Amy drank and felt happy and full. Mom was glad she did not make coffee. From then on, Amy knew to try more new things, like cooking and having fun with her new friends.Once upon a time there was a chubby hedge. Every day it would bloom.

One day it wanted to go on a journey. It wanted to explore somewhere new and exciting. So it started to wander.

The hedgeled out on a small hill, and out over the trees. It was so much fun! But eventually it got too tired to wander anymore. So the hedge stayed in the end for a long time and eventually it found a place to rest.

The hedge was very happy and it thanked the animals for its special help. It was a great adventure for everyone!Once upo

## Advanced

In [115]:
prompts = ["One day, looking over the horizon",
           "In the deep forest of Mirkwood",
           "The sea", 
           "A small bunny jumped in the grass."]
answers = []

for prompt in prompts :
  p_enc = enc.encode(prompt)
  context = torch.tensor(p_enc, dtype=torch.long, device=device).unsqueeze(0)
  generated = model.generate(context, max_new_tokens=100, temperature=1.0, top_k=10)
  answers.append(generated)
  print('\n=== Trained Model Generation ===')
  print(enc.decode(generated[0].tolist()))
  print('=' * 50)


=== Trained Model Generation ===
One day, looking over the horizon and it seemed to be coming true.

The star saw the little girl from the sky, with a sad face, with her sad eyes. The star gave the little girl a kiss, and she smiled heavily. The little girl was so sad that it didn't have a place to stay and she started to cry.

The star and the little girl became friends after that. They played together every day and soon the sun shone brightly. From that day on, every time they felt sad

=== Trained Model Generation ===
In the deep forest of Mirkwood. They are the best of friends. They can spend hours looking for each other. Make friends is so impressed with how smart they are, he can do things that you love.

Pof worth a few moments, Sam and Ben cuddled up in the woods, as the days went over and released. But little did they know, Sam would always be there to sadness him. 

Finally, one day, Sam and Ben started to play in the garden. Sam was still a

=== Trained Model Generation ===

##

In [116]:
# save losses to csv
df = pd.DataFrame(data={"iteration":np.array(iterations, dtype=float), "train_loss":np.array(train_losses, dtype=float), "test_loss":np.array(val_losses, dtype=float)})
df.to_csv(drive_path + f"{random_n_model}_losses.csv")

In [117]:
# save training parameters to file
with open(drive_path+f'training_config_{random_n_model}.txt', 'w') as f:
    f.write(f"Trained model {random_n_model} for {config.max_iters} iterations.\n\
            Batch size : {config.batch_size}\n\
            Block size : {config.block_size}\n\
            Data fraction : {config.data_fraction}\n\
            =============================================\n\
            d_model : {config.d_model}\n\
            n_heads : {config.n_heads}\n\
            n_layers : {config.n_layers}\n\
            d_ff : {config.d_ff}\n\
            dropout : {config.dropout}\n\
            eval_iters : {config.eval_iters}\n")